In [ ]:
# Simple GAN for MNIST (minimal working version — 1 epoch demo)
import tensorflow as tf
import numpy as np

# Generator
def make_generator():
    model = tf.keras.Sequential([
        layers.Dense(128, input_dim=100, activation='relu'),
        layers.Dense(256, activation='relu'),
        layers.Dense(28*28, activation='tanh')
    ])
    return model

# Discriminator
def make_discriminator():
    model = tf.keras.Sequential([
        layers.Dense(256, input_dim=28*28, activation='relu'),
        layers.Dense(128, activation='relu'),
        layers.Dense(1, activation='sigmoid')
    ])
    return model

# Prepare real data
(x_train, _), _ = tf.keras.datasets.mnist.load_data()
x_train = x_train.reshape(-1, 28*28).astype('float32') / 127.5 - 1

# Models
generator = make_generator()
discriminator = make_discriminator()

# Compile
discriminator.compile(optimizer='adam', loss='binary_crossentropy')
gan = tf.keras.Sequential([generator, discriminator])
discriminator.trainable = False
gan.compile(optimizer='adam', loss='binary_crossentropy')

# Training loop (1 epoch for demo)
batch_size = 64
steps = x_train.shape[0] // batch_size

for step in range(steps):
    idx = np.random.randint(0, x_train.shape[0], batch_size)
    real_imgs = x_train[idx]
    noise = np.random.normal(0, 1, (batch_size, 100))
    fake_imgs = generator(noise)

    # Train discriminator
    d_loss_real = discriminator.train_on_batch(real_imgs, np.ones((batch_size, 1)) * 0.9)
    d_loss_fake = discriminator.train_on_batch(fake_imgs, np.zeros((batch_size, 1)) + 0.1)

    # Train generator
    noise = np.random.normal(0, 1, (batch_size, 100))
    g_loss = gan.train_on_batch(noise, np.ones((batch_size, 1)))

print("GAN training (1 epoch) complete. Generator can now produce digits.")